# ETL1

## Funciones auxiliares

Con el fin de estandarizar el tipo de datos que se tiene por columna se definen funcione auxiliares que convierten a tipo numerico, string, int, float, datetime, bit y una función que verifica que no haya ningun id nulo.

In [1]:
import sqlite3
import pandas as pd
import sys 
import io # Para capturar la salida de df.info()

# --- Configuración ---
sqlite_db_file = 'nba.sqlite'

# --- Funciones Auxiliares (COMPLETAS) ---

def safe_to_numeric(series, errors='coerce'):
    series = series.replace(r'^\s*$', pd.NA, regex=True)
    return pd.to_numeric(series, errors=errors)

def convert_to_string(df, columns):
    for col in columns:
        if col in df.columns:
            df[col] = df[col].astype(str).replace({
                'None': pd.NA, 'nan': pd.NA, '<NA>': pd.NA
            }, regex=False)
            df[col] = df[col].str.replace(r'\.0$', '', regex=True)
    return df

def convert_to_int64(df, columns):
    for col in columns:
        if col in df.columns: df[col] = safe_to_numeric(df[col]).astype('Int64')
    return df

def convert_to_float(df, columns):
    for col in columns:
        if col in df.columns: df[col] = safe_to_numeric(df[col]).astype('float64')
    return df

def convert_to_datetime(df, columns):
    for col in columns:
        if col in df.columns: df[col] = pd.to_datetime(df[col], errors='coerce')
    return df

def convert_to_bit(df, columns):
    for col in columns:
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip().str.lower().map({
                '1': 1, 'true': 1, 't': 1, '0': 0, 'false': 0, 'f': 0
            }).fillna(pd.NA).astype('Int64')
    return df

def check_pk_not_null(df, pk_columns):
    pk_list = [pk_columns] if isinstance(pk_columns, str) else list(pk_columns)
    for col in pk_list:
        if col in df.columns and df[col].isnull().any():
            print(f"   🚨 ¡ADVERTENCIA! La columna PK '{col}' contiene NULOS. ¡Esto podría causar problemas al cargar!")            
    return df



## Funciones de limpieza

Para cada una de las tablas se hace un filtrado y limpieza, se tiene en cuenta que para la tablas que no son estaticas sino transaccionales como play_by_play se filtra para las últimas 5 temporadas.

In [2]:
def limpiar_team(df):
    print("   🧹 Limpiando Team...")
    df = convert_to_string(df, ['id'])
    df = convert_to_int64(df, ['year_founded'])
    df = check_pk_not_null(df, 'id')
    return df

def limpiar_team_details(df):
    print("   🧹 Limpiando team_details...")
    df = convert_to_string(df, ['team_id'])
    df = convert_to_int64(df, ['yearfounded', 'arenacapacity'])
    df = check_pk_not_null(df, 'team_id')
    return df

def limpiar_draft_history(df):
    print("   🧹 Limpiando draft_history...")
    str_ids = ['person_id', 'team_id']
    int_cols = ['season', 'round_number', 'round_pick', 'overall_pick']
    bit_cols = ['player_profile_flag']
    if '_rowid' in df.columns: int_cols.append('_rowid')
    df = convert_to_string(df, str_ids)
    df = convert_to_int64(df, int_cols)
    df = convert_to_bit(df, bit_cols)
    df = check_pk_not_null(df, ['person_id', 'season'])
    return df

def limpiar_game(df):
    print("   🧹 Limpiando game...")
    str_ids = ['game_id', 'team_id_home', 'team_id_away']
    int_cols = ['fgm_home', 'fga_home', 'fg3m_home', 'fg3a_home', 'ftm_home', 'fta_home', 'oreb_home', 'dreb_home', 'reb_home', 'ast_home', 'stl_home', 'blk_home', 'tov_home', 'pf_home', 'pts_home', 'plus_minus_home', 'fgm_away', 'fga_away', 'fg3m_away', 'fg3a_away', 'ftm_away', 'fta_away', 'oreb_away', 'dreb_away', 'reb_away', 'ast_away', 'stl_away', 'blk_away', 'tov_away', 'pf_away', 'pts_away', 'plus_minus_away']
    float_cols = ['min', 'fg_pct_home', 'fg3_pct_home', 'ft_pct_home', 'fg_pct_away', 'fg3_pct_away', 'ft_pct_away']
    bit_cols = ['wl_home', 'video_available_home', 'wl_away', 'video_available_away']
    df = convert_to_string(df, str_ids)
    df = convert_to_int64(df, int_cols)
    df = convert_to_float(df, float_cols)
    df = convert_to_bit(df, bit_cols)
    df = convert_to_datetime(df, ['game_date'])
    df = check_pk_not_null(df, 'game_id')
    return df

Se hace la limpieza (quitar filas vacias, estandarizar formatos) de las tablas team, game, team_details y draft history.

In [4]:
# --- Diccionario de Tablas a Procesar ---
tables_to_process = {
    'team': (('id'), limpiar_team),
    'game': (('game_id'), limpiar_game),
    'team_details': (('team_id'), limpiar_team_details),
    'draft_history': (('person_id', 'season'), limpiar_draft_history),
}

# --- Orden de Carga (Respetando Dependencias Lógicas) ---
load_order = [
    'team', 
    'game', 
    'team_details', 
    'draft_history'
]

# --- Proceso Principal ET ---

sqlite_conn = None

print(f"--- 🚀 Iniciando Proceso ET Tablas Seleccionadas ---")

try:
    # --- Conectar a SQLite ---
    print(f"🔌 Conectando a SQLite: {sqlite_db_file}...")
    sqlite_conn = sqlite3.connect(sqlite_db_file)
    
    # --- Bucle para Procesar Cada Tabla en Orden ---
    for table_name in load_order:
        pk_info, clean_func = tables_to_process[table_name]

        print(f"\n=======================================================")
        print(f"--- ⏳ Procesando Tabla: {table_name} ---")
        print(f"=======================================================")
        
        # 1. EXTRAER
        print(f"   📊 Extrayendo datos de '{table_name}'...")
        try:
            # Usamos pd.read_sql_query para leer toda la tabla
            df_sqlite = pd.read_sql_query(f"SELECT * FROM \"{table_name}\"", sqlite_conn)
            if df_sqlite.empty:
                print(f"   ⚠️ No se encontraron datos para '{table_name}'. Saltando...")
                continue
            rows_before = len(df_sqlite)
            print(f"   ✅ Se extrajeron {rows_before} filas.")
        except Exception as e:
            print(f"   ❌ Error al extraer '{table_name}': {e}. Saltando...")
            continue

        # 2. TRANSFORMAR


        df_cleaned = clean_func(df_sqlite.copy()) # Aplicamos limpieza
        #Exportar a csv
        df_cleaned.to_csv(f"{table_name}_limpio.csv", index=False)

        # 3. ELIMINAR FILAS VACÍAS
        rows_before_drop = len(df_cleaned)
        df_cleaned.dropna(how='all', inplace=True)
        rows_after_drop = len(df_cleaned)
        rows_removed = rows_before_drop - rows_after_drop
        if rows_removed > 0:
            print(f"   🗑️ Se eliminaron {rows_removed} filas completamente vacías.")
        
        print(f"   ✨ {table_name} procesada. {rows_after_drop} filas listas.")

        # 4. MOSTRAR MUESTRA
        print("\n   --- MUESTRA DE DATOS LIMPIOS (Primeras 3 filas) ---")
        print(df_cleaned.head(3).to_string())

except Exception as e:
    print(f"❌ Ocurrió un error general durante el proceso: {e}")

finally:
    # --- Cerrar Conexión ---
    if sqlite_conn:
        sqlite_conn.close()
        print("\n\n🚪 Conexión a SQLite cerrada.")

--- 🚀 Iniciando Proceso ET Tablas Seleccionadas ---
🔌 Conectando a SQLite: nba.sqlite...

--- ⏳ Procesando Tabla: team ---
   📊 Extrayendo datos de 'team'...
   ✅ Se extrajeron 30 filas.
   🧹 Limpiando Team...
   ✨ team procesada. 30 filas listas.

   --- MUESTRA DE DATOS LIMPIOS (Primeras 3 filas) ---
           id            full_name abbreviation   nickname       city          state  year_founded
0  1610612737        Atlanta Hawks          ATL      Hawks    Atlanta        Georgia          1949
1  1610612738       Boston Celtics          BOS    Celtics     Boston  Massachusetts          1946
2  1610612739  Cleveland Cavaliers          CLE  Cavaliers  Cleveland           Ohio          1970

--- ⏳ Procesando Tabla: game ---
   📊 Extrayendo datos de 'game'...
   ✅ Se extrajeron 65698 filas.
   🧹 Limpiando game...
   ✨ game procesada. 65698 filas listas.

   --- MUESTRA DE DATOS LIMPIOS (Primeras 3 filas) ---
  season_id team_id_home team_abbreviation_home           team_name_home     ga

## Limpieza play_by_play últimas 5 temporadas

Se calcula los años de las ultimas 5 temporadas de la NBA para los datos disponibles, y se muestra un resumen de la tabla procesada.

In [5]:
# --- Configuración ---
sqlite_db_file = 'nba.sqlite'
table_to_process = 'play_by_play'
game_info_table = 'game_info' # Usaremos esta tabla para las fechas/años

# --- Nombres de columnas ASUMIDOS en game_info (¡AJUSTA SI ES NECESARIO!) ---
# Si tus columnas se llaman diferente, cámbialas aquí:
GAME_INFO_ID_COLUMN = 'game_id' # Columna de ID de juego en game_info
GAME_INFO_DATE_COLUMN = 'game_date' # Columna de fecha en game_info

def limpiar_play_by_play(df):
    """Limpia y convierte tipos para la tabla play_by_play."""
    print("   🧹 Limpiando play_by_play...")
    # Asumimos que la columna GAME_INFO_ID_COLUMN se mapea a 'game_id' en play_by_play
    # o que tienen el mismo nombre. Ajusta si 'game_id' en PBP tiene otro nombre.
    str_ids = [
        'game_id', 'player1_id', 'player1_team_id', 
        'player2_id', 'player2_team_id', 'player3_id', 'player3_team_id'
    ]
    # Si GAME_INFO_ID_COLUMN no es 'game_id', y 'game_id' existe en df, se limpiará.
    # Si GAME_INFO_ID_COLUMN es el nombre usado en df_sqlite, no hay problema.
    
    int_cols = [
        'eventnum', 'eventmsgtype', 'eventmsgactiontype', 'period',
        'person1type', 'person2type', 'person3type'
    ]
    bit_cols = ['video_available_flag']
    df = convert_to_string(df, str_ids)
    df = convert_to_int64(df, int_cols)
    df = convert_to_bit(df, bit_cols)
    df = check_pk_not_null(df, ['game_id', 'eventnum'])
    return df

# --- Proceso Principal ET ---

sqlite_conn = None

print(f"--- 🚀 Iniciando Proceso ET para: {table_to_process} (Temporadas Dinámicas desde {game_info_table}) ---")

try:
    # --- 1. EXTRAER ---
    print(f"🔌 Conectando a SQLite: {sqlite_db_file}...")
    sqlite_conn = sqlite3.connect(sqlite_db_file)
    
    # === PASO 1.1: OBTENER AÑOS DE TEMPORADA DESDE game_info ===
    print(f"   📊 Obteniendo fechas de juego desde '{game_info_table}' para determinar temporadas...")
    try:
        # Leemos solo las columnas necesarias de game_info
        game_info_query = f"SELECT \"{GAME_INFO_ID_COLUMN}\" AS game_id, \"{GAME_INFO_DATE_COLUMN}\" AS game_date FROM {game_info_table}"
        df_game_info = pd.read_sql_query(game_info_query, sqlite_conn)
    except Exception as e:
        print(f"   ❌ Error al leer la tabla '{game_info_table}'. Verifica el nombre y las columnas.")
        print(f"   Error: {e}")
        sys.exit()

    if df_game_info.empty:
        print(f"   ❌ La tabla '{game_info_table}' está vacía o no se pudieron leer datos. No se puede continuar.")
        sys.exit()

    # Convertimos la columna de fecha y derivamos el año de inicio de la temporada
    df_game_info['game_date'] = pd.to_datetime(df_game_info['game_date'], errors='coerce')
    df_game_info.dropna(subset=['game_date'], inplace=True) # Eliminamos filas sin fecha válida

    if df_game_info.empty:
        print(f"   ❌ No hay fechas válidas en '{GAME_INFO_DATE_COLUMN}' en '{game_info_table}'. No se puede continuar.")
        sys.exit()
        
    # Una temporada NBA (ej: 2023-24) usualmente empieza en Oct del primer año.
    # Por lo tanto, si mes < Octubre, pertenece a la temporada que empezó el año anterior.
    df_game_info['season_start_year'] = df_game_info['game_date'].dt.year - (df_game_info['game_date'].dt.month < 10).astype(int)
    
    unique_season_years = sorted(df_game_info['season_start_year'].unique(), reverse=True)
    last_5_season_years = tuple(unique_season_years[:5])

    if not last_5_season_years:
        print("   ❌ No se pudieron determinar las últimas 5 temporadas desde game_info. No se puede continuar.")
        sys.exit()
    print(f"   ✅ Últimos 5 años de inicio de temporada determinados: {last_5_season_years}")
    
    # Paso 1.2: Obtener game_ids para ESAS 5 temporadas/años
    game_ids_to_fetch_df = df_game_info[df_game_info['season_start_year'].isin(last_5_season_years)]
    game_ids_list = game_ids_to_fetch_df['game_id'].astype(str).unique().tolist()

    if not game_ids_list:
        print("   ⚠️ No se encontraron juegos para las temporadas/años especificados. Finalizando.")
        sys.exit()
    print(f"   ✅ Se encontraron {len(game_ids_list)} game_ids para procesar.")

    # Paso 1.3: Obtener datos de play_by_play para esos game_ids
    print(f"   📊 Extrayendo datos de '{table_to_process}' (puede tardar un poco)...")
    placeholders = ', '.join(['?'] * len(game_ids_list))
    # Asegúrate que la columna 'game_id' en play_by_play coincida con GAME_INFO_ID_COLUMN si usas este filtro
    pbp_query = f"SELECT * FROM \"{table_to_process}\" WHERE game_id IN ({placeholders})" 
    
    df_sqlite = pd.read_sql_query(pbp_query, sqlite_conn, params=game_ids_list)

    if df_sqlite.empty:
        print(f"   ⚠️ No se encontraron datos de Play-by-Play para los {len(game_ids_list)} juegos. Finalizando.")
        sys.exit()
        
    rows_before = len(df_sqlite)
    print(f"   ✅ Se extrajeron {rows_before} filas de Play-by-Play.")

    # 2. TRANSFORMAR

    df_cleaned = limpiar_play_by_play(df_sqlite.copy())


    # 3. ELIMINAR FILAS VACÍAS
    rows_before_drop = len(df_cleaned)
    df_cleaned.dropna(how='all', inplace=True)
    rows_after_drop = len(df_cleaned)
    rows_removed = rows_before_drop - rows_after_drop

    # Exportar a csv
    df_cleaned.to_csv(f"{table_to_process}_limpio.csv", index=False)
    
    if rows_removed > 0:
        print(f"   🗑️ Se eliminaron {rows_removed} filas completamente vacías.")
    
    print(f"   ✨ {table_to_process} procesada. {rows_after_drop} filas listas.")

    # 4. MOSTRAR MUESTRA
    print("\n   --- MUESTRA DE DATOS LIMPIOS (Primeras 3 filas) ---")
    print(df_cleaned.head(3).to_string())

except sqlite3.Error as e:
    print(f"❌ Error de SQLite: {e}")
except KeyError as e:
    print(f"❌ Error de Columna: ¡La columna {e} no se encontró como se esperaba! Verifica los nombres en `GAME_INFO_ID_COLUMN` o `GAME_INFO_DATE_COLUMN` o en tu tabla SQLite.")
except Exception as e:
    print(f"❌ Ocurrió un error general durante el proceso: {e}")

finally:
    # --- Cerrar Conexión ---
    if sqlite_conn:
        sqlite_conn.close()
        print("\n\n🚪 Conexión a SQLite cerrada.")

--- 🚀 Iniciando Proceso ET para: play_by_play (Temporadas Dinámicas desde game_info) ---
🔌 Conectando a SQLite: nba.sqlite...
   📊 Obteniendo fechas de juego desde 'game_info' para determinar temporadas...
   ✅ Últimos 5 años de inicio de temporada determinados: (np.int64(2022), np.int64(2021), np.int64(2020), np.int64(2019), np.int64(2018))
   ✅ Se encontraron 5855 game_ids para procesar.
   📊 Extrayendo datos de 'play_by_play' (puede tardar un poco)...
   ✅ Se extrajeron 2494547 filas de Play-by-Play.
   🧹 Limpiando play_by_play...
   ✨ play_by_play procesada. 2494547 filas listas.

   --- MUESTRA DE DATOS LIMPIOS (Primeras 3 filas) ---
      game_id  eventnum  eventmsgtype  eventmsgactiontype  period wctimestring pctimestring                               homedescription                 neutraldescription                visitordescription score scoremargin  person1type player1_id      player1_name player1_team_id player1_team_city player1_team_nickname player1_team_abbreviation  per

# EDA

Se verifican los tipos de datos que lee pandas de sqlite vs los tipos de datos de la base de datos NBA_HoopVision con el fin de identificar incompatibilidades y mejorar el proceso de ETL

In [2]:
import sqlite3
import pandas as pd
from sqlalchemy import create_engine, inspect
import urllib
import sys

# --- Configuración ---
sqlite_db_file = 'nba.sqlite'

sql_server_driver = 'ODBC Driver 17 for SQL Server' 
sql_server_name = 'IAMGROOT'
sql_server_db = 'NBA_HoopVision'

# --- Conexión SQL Server ---
print(f"🔌 Creando conexión a SQL Server: {sql_server_name}/{sql_server_db}...")
try:
    params = urllib.parse.quote_plus(
        f'DRIVER={{{sql_server_driver}}};'
        f'SERVER={sql_server_name};'
        f'DATABASE={sql_server_db};'
        f'Trusted_Connection=yes;'
    )
    sql_engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")
    sql_inspector = inspect(sql_engine)
except Exception as e:
    print(f"❌ Error fatal al conectar a SQL Server: {e}")
    sys.exit()

# --- Funciones de Inspección ---

def get_sqlite_pandas_types(db_path):
    """Lee todas las tablas de SQLite y devuelve sus Dtypes de Pandas."""
    schemas = {}
    conn = None
    print(f"\n--- 🕵️‍♂️ Inspeccionando SQLite ({db_path}) ---")
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = [row[0] for row in cursor.fetchall()]
        print(f"Tablas encontradas: {len(tables)}")

        for table_name in tables:
            try:
                # Leemos solo 1 fila para inferir tipos rápidamente
                df = pd.read_sql_query(f"SELECT * FROM \"{table_name}\" LIMIT 1", conn)
                schemas[table_name] = {col: str(dtype) for col, dtype in df.dtypes.items()}
            except Exception as e:
                print(f"   - ⚠️ No se pudo leer la tabla '{table_name}': {e}")
                schemas[table_name] = {"Error": str(e)}
        
    except Exception as e:
        print(f"❌ Error al inspeccionar SQLite: {e}")
    finally:
        if conn:
            conn.close()
    return schemas

def get_sql_server_types(inspector):
    """Lee todas las tablas de SQL Server y devuelve sus tipos."""
    schemas = {}
    print(f"\n--- 🕵️‍♀️ Inspeccionando SQL Server ({sql_server_db}) ---")
    try:
        schemas_db = inspector.get_schema_names()
        # A menudo nos interesa 'dbo', pero busquemos en todas por si acaso
        print(f"Esquemas encontrados: {schemas_db}")
        
        for schema in schemas_db:
             # Omitimos esquemas del sistema
            if schema in ['guest', 'INFORMATION_SCHEMA', 'sys', 'db_owner', 
                          'db_accessadmin', 'db_securityadmin', 'db_ddladmin', 
                          'db_backupoperator', 'db_datareader', 'db_datawriter',
                          'db_denydatareader', 'db_denydatawriter']:
                continue

            tables = inspector.get_table_names(schema=schema)
            print(f"Tablas encontradas en '{schema}': {len(tables)}")
            for table_name in tables:
                full_table_name = f"{schema}.{table_name}"
                try:
                    columns = inspector.get_columns(table_name, schema=schema)
                    schemas[table_name] = {col['name']: str(col['type']) for col in columns}
                except Exception as e:
                    print(f"   - ⚠️ No se pudo leer la tabla '{full_table_name}': {e}")
                    schemas[table_name] = {"Error": str(e)}

    except Exception as e:
        print(f"❌ Error al inspeccionar SQL Server: {e}")
    return schemas

def analyze_mismatch(pandas_type, sql_type):
    if not pandas_type or not sql_type:
        return "FALTA DATO"

    p_type = str(pandas_type).lower()
    # Para SQL Server, tomamos el tipo base (ej. 'NVARCHAR' de 'NVARCHAR(255)')
    s_type_full = str(sql_type).upper()
    s_type_base = s_type_full.split('(')[0] 

    # Casos OK (comunes)
    if p_type.startswith('int') and s_type_base in ['INT', 'INTEGER', 'BIGINT', 'SMALLINT', 'TINYINT']:
        return "✅ OK"
    if p_type.startswith('float') and s_type_base in ['FLOAT', 'REAL', 'DECIMAL', 'NUMERIC']:
        return "✅ OK"
    if p_type == 'object' and s_type_base in ['VARCHAR', 'NVARCHAR', 'TEXT', 'CHAR', 'NCHAR']:
        return "✅ OK"
    if p_type == 'bool' and s_type_base == 'BIT':
        return "✅ OK"
    if p_type.startswith('datetime64') and s_type_base in ['DATE', 'DATETIME', 'DATETIME2', 'SMALLDATETIME', 'TIME']:
        return "✅ OK"

    # Focos Rojos (comunes)
    if p_type.startswith('float') and s_type_base in ['INT', 'INTEGER', 'BIGINT', 'SMALLINT', 'TINYINT']:
        return f"🚨 ¡{p_type.upper()} vs {s_type_base}! (Necesita Int64 nullable)"
    if p_type == 'object' and s_type_base in ['INT', 'INTEGER', 'BIGINT', 'SMALLINT', 'TINYINT', 'FLOAT', 'REAL', 'DECIMAL', 'NUMERIC', 'BIT', 'DATE', 'DATETIME', 'DATETIME2']:
        return f"🚨 ¡OBJECT vs {s_type_base}! (Necesita conversión específica)"
    if p_type.startswith('int') and s_type_base == 'BIT':
        return f"⚠️ {p_type.upper()} vs BIT (¿Se mapea 0/1?)"

    # Casos menos comunes o que requieren inspección
    if p_type == 'object' and s_type_base not in ['VARCHAR', 'NVARCHAR', 'TEXT', 'CHAR', 'NCHAR']:
        return f"❓ OBJECT vs {s_type_base} (Revisar si es texto o necesita conversión)"

    return "❓ Revisar (Combinación no cubierta)"

# --- Ejecución y Comparación ---

sqlite_types = get_sqlite_pandas_types(sqlite_db_file)
sql_server_types = get_sql_server_types(sql_inspector)

print("\n\n--- 📊 Comparación de Esquemas ---")

# Usamos las tablas de SQLite como base para la comparación
all_tables = sorted(list(sqlite_types.keys()))

for table in all_tables:
    print(f"\n=======================================")
    print(f" Tabla: {table}")
    print(f"=======================================")
    
    sqlite_cols = sqlite_types.get(table, {})
    sql_cols = sql_server_types.get(table, {})

    if "Error" in sqlite_cols:
        print(f"  - Error en SQLite: {sqlite_cols['Error']}")
        continue
    if not sql_cols or "Error" in sql_cols:
        print(f"  - ⚠️ ¡Tabla NO encontrada o con Error en SQL Server!")
        continue

    all_cols = sorted(list(sqlite_cols.keys()))

    print(f"  {'Columna':<30} | {'SQLite (Pandas)':<15} | {'SQL Server':<15} | {'Análisis':<30}")
    print(f"  {'-'*30} | {'-'*15} | {'-'*15} | {'-'*30}")

    for col in all_cols:
        p_type = sqlite_cols.get(col, 'N/A')
        s_type = sql_cols.get(col, 'N/A')
        analysis = analyze_mismatch(p_type, s_type)
        print(f"  {col:<30} | {p_type:<15} | {s_type:<15} | {analysis}")

    # Verificamos si hay columnas en SQL Server que no están en SQLite
    missing_in_sqlite = [c for c in sql_cols.keys() if c not in sqlite_cols]
    if missing_in_sqlite:
        print("\n  Columnas ADICIONALES en SQL Server:")
        for col in missing_in_sqlite:
            print(f"    - {col} ({sql_cols[col]})")

print("\n--- Comparación Finalizada ---")

# Cerramos el motor de SQL Server
if sql_engine:
    sql_engine.dispose()
    print("🚪 Conexión a SQL Server cerrada.")

🔌 Creando conexión a SQL Server: IAMGROOT/NBA_HoopVision...

--- 🕵️‍♂️ Inspeccionando SQLite (nba.sqlite) ---
Tablas encontradas: 16

--- 🕵️‍♀️ Inspeccionando SQL Server (NBA_HoopVision) ---
Esquemas encontrados: ['db_accessadmin', 'db_backupoperator', 'db_datareader', 'db_datawriter', 'db_ddladmin', 'db_denydatareader', 'db_denydatawriter', 'db_owner', 'db_securityadmin', 'dbo', 'guest', 'INFORMATION_SCHEMA', 'sys']
Tablas encontradas en 'dbo': 11


--- 📊 Comparación de Esquemas ---

 Tabla: common_player_info
  Columna                        | SQLite (Pandas) | SQL Server      | Análisis                      
  ------------------------------ | --------------- | --------------- | ------------------------------
  birthdate                      | object          | DATE            | 🚨 ¡OBJECT vs DATE! (Necesita conversión específica)
  country                        | object          | NVARCHAR(100) COLLATE "SQL_Latin1_General_CP1_CI_AS" | ✅ OK
  display_fi_last                | object  